In [ ]:
!pip install selenium pandas webdriver-manager

import sys
!{sys.executable} -m pip install selenium pandas webdriver-manager

import selenium
import pandas
from webdriver_manager.chrome import ChromeDriverManager

print(f"Selenium version: {selenium.__version__}")
print(f"Pandas version: {pandas.__version__}")
print("Webdriver Manager is installed.")
print("All required libraries are installed.")

Selenium version: 4.22.0
Pandas version: 2.0.3
Webdriver Manager is installed.
All required libraries are installed.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

if not os.path.exists("election_results"):
    os.makedirs("election_results")

driver.get("https://results.eci.gov.in/PcResultGenJune2024/index.htm")

def extract_table_data():
    results_table = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "table-responsive"))
    )
    rows = results_table.find_elements(By.TAG_NAME, "tr")
    party_results = []
    for row in rows[1:]:
        cols = row.find_elements(By.TAG_NAME, "td")
        if cols:
            party_results.append({
                'S.No': cols[0].text.strip(),
                'Candidate': cols[1].text.strip(),
                'Party': cols[2].text.strip(),
                'EVM Votes': cols[3].text.strip(),
                'Postal Votes': cols[4].text.strip(),
                'Total Votes': cols[5].text.strip(),
                'Percentage of votes': cols[6].text.strip()
            })
    return party_results

try:
    state_dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_Result1_ddlState"]'))
    )
    state_dropdown.click()

    state_options = state_dropdown.find_elements(By.TAG_NAME, "option")[1:]

    for i, state_option in enumerate(state_options, start=1):
        state_name = state_option.text
        state_option.click()
        time.sleep(2)

        constituency_dropdown = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_Result1_ddlState"]'))
        )
        constituency_dropdown.click()

        constituency_options = constituency_dropdown.find_elements(By.TAG_NAME, "option")[1:]

        for j, constituency_option in enumerate(constituency_options, start=1):
            constituency_name = constituency_option.text
            constituency_option.click()
            time.sleep(2)

            results_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/main/div/div[2]/div[2]/ul/li[2]/a/img'))
            )
            results_link.click()
            time.sleep(2)

            party_results = extract_table_data()

            df = pd.DataFrame(party_results)
            file_name = f"election_results/{state_name}_{constituency_name}.csv"
            df.to_csv(file_name, index=False)
            print(f"Saved results for {state_name} - {constituency_name}")

            driver.back()
            time.sleep(2)

        driver.get("https://results.eci.gov.in/PcResultGenJune2024/index.htm")
        time.sleep(2)

        state_dropdown = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="ctl00_ContentPlaceHolder1_Result1_ddlState"]'))
        )
        state_dropdown.click()
        state_options = state_dropdown.find_elements(By.TAG_NAME, "option")[1:]

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    driver.quit()

print("Scraping completed.")

AttributeError: 'NoneType' object has no attribute 'split'